# Merging Behavioral and Neural TMS Effects (via cVAE Embeddings)

## Day 2 TMS Effect
- **Behavioral:**  
  Difference between **sham–sham** and **sham–cTBS** conditions on Day 2.  
- **Neural (MRI):**  
  Selected Day 2 sessions where **Day 1 was sham**.  
  Computed the difference in `distance_to_null` between Day 2 sessions following **cTBS vs. sham**.

## Day 1 TMS Effect
- **Behavioral:**  
  Same computation logic as above, but applied to Day 1 conditions.  
- **Neural (MRI):**  
  Selected Day 1 sessions where **Day 2 was sham**.  
  Computed the difference in `distance_to_null` between **cTBS vs. sham** sessions.

In [25]:
from scipy.stats import pearsonr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [26]:
# Load the session distance data
use_roi = 3  # simply run this code with 0,1,2,3 (did not write loop)
output_filename = f"../../data_mri_processed/session_distances_roi_{use_roi}.csv"
df = pd.read_csv(output_filename)

### compute day 2 tms effect from MRI data

In [27]:
# List to store final results for each subject
results = []

# Loop through each unique subject
for subject_id in df["subject"].unique():
    # Extract data only for this subject
    df_sub = df[df["subject"] == subject_id]

    # Prepare a dictionary to store paired sessions by prefix (e.g., S1, S2, S3)
    # Each prefix will map to a dict with keys "D1" and "D2"
    session_pairs = {}

    # Go through each row and organize sessions into SxD1 and SxD2
    for _, row in df_sub.iterrows():
        session_label = row["session"]   # e.g., S1D1
        prefix = session_label[:2]       # e.g., S1
        day = session_label[2:]          # e.g., D1

        # Initialize dictionary if this is the first time seeing the prefix
        if prefix not in session_pairs:
            session_pairs[prefix] = {}

        # Store the row (a Series) under the corresponding day
        session_pairs[prefix][day] = row

    # Variables to hold the Day 2 distance_to_null values for:
    
    sham_sham_val = None
    sham_cTBS_val = None

    # Loop through each session pair (S1, S2, S3...) for this subject
    for prefix, pair in session_pairs.items():
        # Try to get both D1 and D2 rows
        d1 = pair.get("D1", None)
        d2 = pair.get("D2", None)

        # Only proceed if both Day 1 and Day 2 exist
        if d1 is not None and d2 is not None:
            # Case 1: Day 1 was Sham, and Day 2 was also Sham → sham-sham baseline
            if d1["condition"] == "S" and d2["condition"] == "S":
                sham_sham_val = d2["distance_to_null"]

            # Case 2: Day 1 was Sham, and Day 2 was TMS (C) → sham-cTBS effect
            elif d1["condition"] == "S" and d2["condition"] == "C":
                sham_cTBS_val = d2["distance_to_null"]

    # Compute the Day 2 TMS effect if both values were found
    if sham_sham_val is not None and sham_cTBS_val is not None:
        tms_effect = sham_cTBS_val - sham_sham_val
    else:
        tms_effect = None  # Leave empty if we can't compute for this subject

    # Save the result
    results.append({
        "subject": subject_id,
        "day2_tms_effect_mri": tms_effect
    })

# Convert results into a DataFrame
df_tms_effect_day2 = pd.DataFrame(results)
df_tms_effect_day2.head()

,subject,day2_tms_effect_mri
0,NODEAP_06,-0.459559
1,NODEAP_07,-1.252594
2,NODEAP_08,-1.876382
3,NODEAP_09,-1.571389
4,NODEAP_10,-0.818196


In [28]:
# day 2 tms effect
df_behavior = df_tms_effect_day2.copy()

# Behavioral effect summary
df_effect = pd.read_csv("../../beh_data_processed/TMS_effect_summary_day2.csv")

# Clean subject ID columns
df_behavior["subject"] = df_behavior["subject"].str.strip()
df_effect["SubID"] = df_effect["SubID"].str.strip()

# Merge by subject ID
merged_df = df_effect.merge(df_behavior, left_on="SubID", right_on="subject", how="inner")
merged_df.head()

,SubID,StimLoc,TMS_effect_actual,TMS_effect_predicted,subject,day2_tms_effect_mri
0,NODEAP_06,pOFC,0.583333,-0.119540,NODEAP_06,-0.459559
1,NODEAP_08,pOFC,-0.055556,-0.065463,NODEAP_08,-1.876382
2,NODEAP_09,pOFC,0.492063,0.163347,NODEAP_09,-1.571389
3,NODEAP_10,pOFC,0.277778,0.221088,NODEAP_10,-0.818196
4,NODEAP_15,pOFC,0.500000,0.437148,NODEAP_15,-1.246996


In [29]:
# Save merged data to Excel file
output_filename = f"../../data_mri_processed/merged_day2_tms_effect_{use_roi}.csv"
merged_df.to_csv(output_filename, index=False)

### compute day 1 tms effect from MRI data

In [30]:
results = []

for subject_id in df["subject"].unique():
    df_sub = df[df["subject"] == subject_id]
    session_pairs = {}

    for _, row in df_sub.iterrows():
        session_label = row["session"]
        prefix = session_label[:2]  # S1, S2, etc.
        day = session_label[2:]     # D1 or D2

        if prefix not in session_pairs:
            session_pairs[prefix] = {}
        session_pairs[prefix][day] = row

    # These will store Day 1 values for the sessions that qualify
    d1_sham_sham_val = None  # Day 1 = S, Day 2 = S
    d1_tms_sham_val = None   # Day 1 = C, Day 2 = S

    for prefix, pair in session_pairs.items():
        d1 = pair.get("D1", None)
        d2 = pair.get("D2", None)

        if d1 is not None and d2 is not None:
            if d1["condition"] == "S" and d2["condition"] == "S":
                d1_sham_sham_val = d1["distance_to_null"]
            elif d1["condition"] == "C" and d2["condition"] == "S":
                d1_tms_sham_val = d1["distance_to_null"]

    # Compute Day 1 TMS effect (TMS - Sham), both followed by Sham on Day 2
    if d1_sham_sham_val is not None and d1_tms_sham_val is not None:
        tms_effect_day1 = d1_tms_sham_val - d1_sham_sham_val
    else:
        tms_effect_day1 = None

    results.append({
        "subject": subject_id,
        "day1_tms_effect_mri": tms_effect_day1
    })

df_tms_effect_day1 = pd.DataFrame(results)
df_tms_effect_day1.head()


,subject,day1_tms_effect_mri
0,NODEAP_06,-0.428906
1,NODEAP_07,-0.343428
2,NODEAP_08,1.146035
3,NODEAP_09,-0.841440
4,NODEAP_10,-0.956951


In [31]:
df_behavior = df_tms_effect_day1.copy()
df_effect = pd.read_csv("../../beh_data_processed/TMS_effect_summary_day1.csv")

# Clean subject ID columns
df_behavior["subject"] = df_behavior["subject"].str.strip()
df_effect["SubID"] = df_effect["SubID"].str.strip()

# Merge by subject ID
merged_df = df_effect.merge(df_behavior, left_on="SubID", right_on="subject", how="inner")
merged_df.head()

,SubID,StimLoc,TMS_effect_actual,TMS_effect_predicted,subject,day1_tms_effect_mri
0,NODEAP_06,pOFC,0.083333,-0.182800,NODEAP_06,-0.428906
1,NODEAP_08,pOFC,-0.166667,-0.169022,NODEAP_08,1.146035
2,NODEAP_09,pOFC,0.269841,0.029955,NODEAP_09,-0.841440
3,NODEAP_10,pOFC,0.194444,0.038926,NODEAP_10,-0.956951
4,NODEAP_15,pOFC,0.333333,0.383016,NODEAP_15,2.311917


In [32]:
output_filename = f"../../data_mri_processed/merged_day1_tms_effect_{use_roi}.csv"
merged_df.to_csv(output_filename, index=False)